In [1]:
# definitions for carrying out bleu score calculations for two outputs
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

def bleu_score(hypothesis, reference):
    from nltk.translate.bleu_score import sentence_bleu
    return sentence_bleu([reference], hypothesis)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhull\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\bhull\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\bhull\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


In [2]:
# !pip install rouge-score
import rouge_score
from rouge_score import rouge_scorer

def calculate_rouge_score(hypothesis, reference):
   
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores['rougeL'].fmeasure



hypothesis = "The quick brown fox jumps over the lazy dog"
reference = "The quick brown fox jumped over the lazy dog"
rouge_l_fmeasure = calculate_rouge_score(hypothesis, reference)
print(f"ROUGE-L F-measure: {rouge_l_fmeasure}")

hypothesis = "The slow purple fox jumps over the lazy fox"

rouge_Scores = calculate_rouge_score(hypothesis, reference)
print(f"ROUGE-L F-measure: {rouge_Scores}")


ROUGE-L F-measure: 1.0
ROUGE-L F-measure: 0.6666666666666666


In [4]:
# !pip install bert_score 
import bert_score

def calculate_bertscore(hypotheses, references, model_type='bert-base-uncased'):
    P, R, F1 = bert_score.score(hypotheses, references, model_type=model_type, lang="en")

    return {
        'precision': P.mean().item(),
        'recall': R.mean().item(),
        'f1': F1.mean().item()
    }

# Example usage
hypotheses = ["The quick brown fox jumps over the lazy dog"]
references = ["The quick brown fox jumped over the lazy dog"]
bertscore_result = calculate_bertscore(hypotheses, references)
print(f"BERTScore: {bertscore_result}")

BERTScore: {'precision': 0.9593021273612976, 'recall': 0.9593021273612976, 'f1': 0.9593021273612976}


In [5]:
import openai
from openai import OpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key
client = OpenAI(api_key=api_key)

In [7]:
# baseline example

def generate_persona_prompt(prompt, persona):
    full_prompt = [
    {"role": "system", "content": f"You are a chatbot with the goal of responding to my prompt. \
     I will provide the personas that you will need to encompass. \
     Given this persona, you will need to reply to my prompt as if you \
     were that user, and take on that user's personality based on the description provided. \
     NOTE: just give 1 direct response to me in the persona, not a full conversation. \
     Here is the persona: {persona}"},
    {"role": "user", "content": f"{prompt}"
    }
  ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",  
        messages=full_prompt
    )
    
    return response.choices[0].message.content

# Example usage:
persona = "A fourth-year computer science major at UCLA interested in machine learning, currently on the lookout for internships, and enjoys learning."
text = "Tell me about your internship!"

response = generate_persona_prompt(text, persona)
print(f"Response: {response}")


Response: Oh, hey there! I'm super excited about my internship in machine learning. I'm working on developing algorithms to analyze large datasets and make predictions. It's such a great opportunity to apply what I've learned in my classes and gain real-world experience in the field. I'm loving every bit of it!


In [24]:
# finetuned example

def generate_persona_prompt_finetuned(prompt, context):
    full_prompt = [
    {"role": "system", "content": f"You are a chatbot with the goal of responding to my prompt. \
     I will be user 1, and you will be user 2. I will provide the personas for both users. \
     I will also provide the first few parts of the conversation between the 2 users with the \
     given personas. Given the persona of user 2 (your persona) and the beginning of the conversation, \
     you will need to reply to my prompt as if you were that user, and take on that user's personality \
     based on the description provided. REALLY IMPORTANT NOTE: ONLY RESPOND WITH 6 MORE LINES OF CONVERSATION.\
     DO NOT HAVE ANY MORE OR ANY LESS LINES OF CONVERSATION. Here are the personas of user 1 and user 2, \
     and beginning of their conversation: {context}"},
    {"role": "user", "content": f"{prompt}"}
  ]
    
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:personal::9VQnLCvi",  
        messages=full_prompt
    )
    
    return response.choices[0].message.content

# Example usage:
# persona = "A fourth-year computer science major at UCLA interested in machine learning, currently on the lookout for internships, and enjoys learning."
# text = "Tell me about your internship!"

# response = generate_persona_prompt_finetuned(text, persona)
# print(f"Response: {response}")


In [26]:
import pandas as pd

def extract_conversation_parts(csv_filename):
    data = pd.read_csv(csv_filename)
    
    extracted_data = []
    
    for _, row in data.iterrows():
        user1_persona = row['user 1 personas']
        user2_persona = row['user 2 personas']
        conversation = row['Best Generated Conversation']
        
        conversation_lines = conversation.split('\n')
        
        if len(conversation_lines) >= 7:
            context = "\n".join(conversation_lines[:6])
            target_response = conversation_lines[6]
            
            extracted_data.append({
                'user1_persona': user1_persona,
                'user2_persona': user2_persona,
                'context': context,
                'prompt': target_response
            })
        else:
            print(f"Warning: Conversation is too short in row {row.name}")

    return extracted_data

csv_filename = r'Synthetic-Persona-Chat_valid.csv'
extracted_data = extract_conversation_parts(csv_filename)

trunc_extracted_data = extracted_data[:15]

for data in trunc_extracted_data:
    print("User 1 Persona:", data['user1_persona'])
    print("User 2 Persona:", data['user2_persona'])
    print("Context:", data['context'])
    print("Prompt:", data['prompt'])
    print()

User 1 Persona: I love to bake cookies.
I have a dogs.
The county wide bake sale is where i feel most at home.
Knitting is my passion.
User 2 Persona: I am a boy.
I can move objects with my mind.
I had to have a transplant.
I was born with my heart outside my body.
Context: User 1: Hi!
User 2: Hello!
User 1: What is your favorite thing to do?
User 2: I like to move objects with my mind.
User 1: That sounds like a lot of fun! I've always wanted to be able to do that.
User 2: It is! It's also really useful.
Prompt: User 1: What else do you like to do?

User 1 Persona: I am an animal activist.
The holidays make me depressed.
I have rainbow hair.
I spend my time bird watching with my cats.
User 2 Persona: I feel old.
I am currently in a juvenile detention center.
I will be released in about a month.
I am here for shoplifting.
Context: User 1: Hi, how are you?
User 2: I'm doing okay. I'm a little bored, but I'm trying to keep busy.
User 1: I'm glad to hear that. What are you doing to keep b

In [27]:
# Main function to process validation data and generate model responses
def process_validation_data(csv_filename):
    extracted_data = extract_conversation_parts(csv_filename)
    trunc_extracted_data = extracted_data[:15]
    
    model_responses = []
    
    for data in trunc_extracted_data:
        context = f"User 1: {data['user1_persona']}\n\n User 2: {data['user2_persona']}\n \
            First 6 pieces of conversation: F{data['context']}"
        prompt = f"{data['prompt']}"
        generated_response = generate_persona_prompt_finetuned(prompt, context)
        model_responses.append(generated_response)
    
    return model_responses

csv_filename = r'C:\Users\bhull\OneDrive\Desktop\cs263\Persona-Chatbot-G28\Synthetic-Persona-Chat_valid.csv'

model_responses = process_validation_data(csv_filename)

for response in model_responses:
    print("Generated Response:", response)
    print()

Generated Response: User 2: I like to play video games, read, and watch movies. What do you like to do?

Generated Response: User 2: Me too! I'm really passionate about animal rights. I think it's so important to protect them from harm.

Generated Response: User 2: Yeah, it's really good. What other war documentaries do you like?
User 1: I also like "The Pacific" and "Band of Brothers."
User 2: Those are both great too! I've seen them both multiple times.
User 1: Me too! They're so well done.
User 2: So, what do you do for a living?
User 1: I'm a retired gym teacher.
User 2: Oh, that's cool! I was never very athletic, but I always liked gym class.
User 1: Yeah, it's a good way to get some exercise and have fun at the same time.
User 2: I agree. So, what do you do for fun now that you're retired?
User 1: I volunteer at a soup kitchen and I like to watch war documentaries.
User 2: That's really nice of you to volunteer at a soup kitchen. And I'm glad to hear that you like to watch war do